In [ ]:
# !ls text_alignment/susp/

In [ ]:
import xml.etree.ElementTree as ET
import os

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd


model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def predict_pair(text1, text2):
    with torch.inference_mode():
        out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        return {v: proba[k] for k, v in model.config.id2label.items()}['entailment']

In [ ]:
!ls

In [ ]:
data = []
for p in 'manually-paraphrased', 'manually_paraphrased2':
    for f in os.listdir(f'data/raw/paraplag/tasks/{p}/'):
        if not f.endswith('.xml'):
            continue
        task_path = f'data/raw/paraplag/tasks/{p}/{f}'
        task = ET.parse(task_path).getroot()

        susp_name = task.attrib['reference']
        susp_path = f'data/raw/paraplag/susp/{susp_name}'
        susp = open(susp_path).read()

        for elem in list(task):

            source_attribs = elem.attrib
            source_path = f"data/raw/paraplag/src/{source_attribs['source_reference']}"
            source = open(source_path).read()

            so, sl = int(source_attribs['source_offset']), int(source_attribs['source_length'])
            to, tl = int(source_attribs['this_offset']), int(source_attribs['this_length'])
            text_source = source[so:so+sl]
            text_susp = susp[to:to+tl]
            obfuscation = source_attribs['obfuscation']
            id = susp_name.split('.')[0]
            data.append([id, obfuscation, text_susp, text_source])

In [ ]:
len(data)

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns=['cluster_id', 'obfuscation', 'susp', 'src'])

In [ ]:
df[df['src'] == 'Чингис так ловко управлял завоеванной частью Китая, что она не восставала во время его отсутствия.']

In [ ]:
df.obfuscation = df.obfuscation.apply(lambda x: ','.join(sorted(set(x.split(',')))))

In [ ]:
df.obfuscation.value_counts().head(20)

In [ ]:
def unique(it):
    seen = set()
    for elem in it:
        if elem in seen:
            continue
        seen.add(elem)
        yield elem

In [ ]:
a = df.groupby('susp').agg({'src': list, 'obfuscation':lambda x: set(x).pop(), 'cluster_id': lambda x: set(x).pop()}).reset_index()
a = a[a.src.apply(len) > 1]
a = a[a.obfuscation.str.contains('|'.join(['CCT', 'SSP', 'SEP', 'LPR', 'HPR']))]
a.columns = ['text1', 'text2', 'obfuscation', 'cluster_id']
a.text2 = a.text2.apply(lambda x: ' '.join(unique(x)))

In [ ]:
b = df.groupby('src').agg({'susp': list, 'obfuscation':lambda x: set(x).pop(), 'cluster_id': lambda x: set(x).pop()}).reset_index()
b = b[b.susp.apply(len) > 1]
b = b[b.obfuscation.str.contains('|'.join(['CCT', 'SSP', 'SEP', 'LPR', 'HPR']))]
b.columns = ['text2', 'text1', 'obfuscation', 'cluster_id']
b.text1 = b.text1.apply(lambda x: ' '.join(unique(x)))

In [ ]:
df_splt = pd.concat([a,b])

In [ ]:
from razdel import tokenize

def find_same(row):
    toks1 = [_.text.lower() for _ in tokenize(row['text1'])]
    toks2 = [_.text.lower() for _ in tokenize(row['text2'])]
    
    toks1 = [t for t in toks1 if t.isalpha()]
    toks2 = [t for t in toks2 if t.isalpha()]

    return toks1 == toks2
    
# df_splt = df_splt[~df_splt.apply(find_same, axis=1)]

In [ ]:
df_splt['ent1'] = df_splt.apply(lambda x: predict_pair(x['text1'], x['text2']), axis=1)
df_splt['ent2'] = df_splt.apply(lambda x: predict_pair(x['text2'], x['text1']), axis=1)

In [ ]:
ex = tmp.sample(1).iloc[0]

text1 = ex['text1']
text2 = ex['text2']

print(text1)
print(ex['obfuscation'])
print(text2)
print(ex['ent1'])
print(ex['ent2'])
# print(predict_pair(text1, text2))
# print(predict_pair(text2, text1))

In [ ]:
ent_data = []

import tqdm

for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    ent1 = predict_pair(row['susp'], row['src'])
    ent2 = predict_pair(row['src'], row['susp'])
    ent_data.append((i, ent1, ent2))

In [ ]:
df['ent1'] = [e[1] for e in ent_data]
df['ent2'] = [e[2] for e in ent_data]

In [ ]:
df.columns

In [ ]:
df.columns = ['cluster_id', 'obfuscation', 'text1', 'text2', 'ent1', 'ent2']

In [ ]:
para_ent_df = pd.concat([df, df_splt])

In [ ]:
# para_ent_df.to_csv('data/paraplag_entailment.tsv', sep='\t', index=False)

In [1]:
import pandas as pd

para = pd.read_csv('data/paraplag_entailment.tsv', sep='\t')

In [3]:
para = para[(para['ent1'] > 0.6) & (para['ent2'] > 0.6)]

In [4]:
# para = para[~para.apply(find_same, axis=1)]

NameError: name 'find_same' is not defined

In [ ]:
df

In [ ]:
df[df.text2 == 'Все больше и больше людей в наше время хочет научиться программировать.']

In [ ]:
ex = para.sample(1).iloc[0]

text1 = ex['text1']
text2 = ex['text2']

print(text1)
print(ex['obfuscation'])
print(text2)
print(ex['ent1'])
print(ex['ent2'])
# print(predict_pair(text1, text2))
# print(predict_pair(text2, text1))

In [ ]:
para.drop_duplicates('text2')

In [ ]:
text2, text1 = b['susp'], ' '.join(b['src'])
print(predict_pair(text1, text2))
print(text1)
print(set(b['obfuscation']))
print(text2)

In [ ]:
a = df[df.obfuscation.apply(lambda x: len(x.split(','))) > 1].sample(1).iloc[0]
print(a['susp'])
print(a['src'])
print(a['obfuscation'])

In [7]:
def same(s1, s2):
    toks1, toks2 = tokenize(s1), tokenize(s2)
    s1 = ' '.join([_.text.lower() for _ in toks1 if _.text.isalpha()])
    s2 = ' '.join([_.text.lower() for _ in toks2 if _.text.isalpha()])
    if s1 in s2 or s2 in s1:
        return True
#     ls1 = [element for element in toks1 if element in toks2]
#     if ls1 == toks1:
#         return True
#     ls2 = [element for element in toks2 if element in toks1]
#     if ls2 == toks2:
#         return True
#     if toks1 == toks2:
#         return True
    return False

In [8]:
from razdel import tokenize

In [9]:
p = para[~para.apply(lambda x: same(x['text1'], x['text2']), axis=1)]

In [22]:
k = p[p.obfuscation == 'SSP'].sample(1).iloc[0]
print(k['text1'])
print('*'*10)
print(k['text2'])

Под влиянием т Этот тип травмы возникает при контакте человека с источником электроэнергии непосредственном или косвенном .
**********
ЭЛЕКТРОТРАВМА возникает при непосредственном или косвенном контакте человека с источником электроэнергии.


In [23]:
len(p)

3044

In [24]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
from collections import Counter

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def match_ents(s1, s2):
    doc1, doc2 = Doc(s1), Doc(s2)
    
    doc1.segment(segmenter)
    doc2.segment(segmenter)
    
    doc1.tag_ner(ner_tagger)
    doc2.tag_ner(ner_tagger)
    
    if Counter([s.type for s in doc1.spans]) != Counter([s.type for s in doc2.spans]):
        return False
    
    numbers1 = set()
    numbers2 = set()
    
    for tok in doc1.tokens:
        if has_numbers(tok.text):
            numbers1.add(tok.text)
    
    for tok in doc2.tokens:
        if has_numbers(tok.text):
            numbers2.add(tok.text)
    
    if numbers1 != numbers2:
        return False
    
    return True
    

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def match_abbr(s1, s2):
    abbrs1 = set()
    abbrs2 = set()
    
    for tok in tokenize(s1):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs1.add(tok.text)
    for tok in tokenize(s2):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs2.add(tok.text)
    
#     print(abbrs1, abbrs2)
    if abbrs1 == abbrs2:
        return True
    return False

In [25]:
p = p[p.apply(lambda x: match_ents(x['text1'], x['text2']), axis=1)]

In [26]:
p = p[p.apply(lambda x: match_abbr(x['text1'], x['text2']), axis=1)]

In [28]:
p.sample(2)

,cluster_id,obfuscation,text1,text2,ent1,ent2
4016,173,LPR,"Ситуация изменилась в 1997 году, когда вернулс...",Ситуация изменилась с возвращением в 1997 году...,0.962912,0.993793
6675,148,ADD,"В армии же используются свои компасы, наиболее...",В армии наиболее распространены компас системы...,0.970211,0.877271


In [27]:
len(p)

2399

In [234]:
p = p[~p.obfuscation.isin(['DEL', 'ADD'])]

In [239]:
k = p.sample(1).iloc[0]
print(k['text1'])
print(k['text2'])

Этот эффект колебаний обусловлен не более 51 см/c.
Эффект колебаний обусловлен не более 51 сантиметра в секунду.


In [208]:
p[p.obfuscation == 'DEL']

,cluster_id,obfuscation,text1,text2,ent1,ent2
45,127,DEL,в результате перегрева двигателя прогорал пор...,Однако из-за сильного перегрева мотора прогора...,0.747398,0.992827
180,68,DEL,"Возможна ситуация, когда машина не останавлива...","Кроме того, возможна ситуация, когда машина не...",0.639743,0.912762
202,169,DEL,"Как известно, ограниченное количество структур...","Как известно, существует ограниченное количест...",0.630116,0.674603
368,73,DEL,Создание звуковых картин требовало изменения в...,Ведь создание звуковых картин требовало кардин...,0.642114,0.996345
465,214,DEL,"Играли чисто для своего удовольствия, и в осн...","Играли просто для собственного удовольствия, и...",0.856813,0.930801
...,...,...,...,...,...,...
18934,2027,DEL,"Конечно, лидерство в этом обходится ему дорого.","Правда, лидерство в этой области обходится ему...",0.801678,0.994250
19756,2021,DEL,В ходе этой диеты при избавлении от болезни до...,Эта система в случае ликвидации патологическог...,0.857868,0.645023
20353,2039,DEL,Неожиданно из толщи воды поднимается Дагон и о...,"И неожиданно, из толщи воды появляется Дагон и...",0.987206,0.987593
20919,2039,DEL,С восьми лет он начал увлекаться химией и астр...,"В возрасте восьми лет он увлекся наукой, снача...",0.817078,0.976843


In [226]:
p.obfuscation.value_counts().head(20)

LPR            796
HPR            539
ADD            294
DEL            221
SSP            102
CCT             79
SYN             32
SHF,SYN         20
LPR,SYN         15
LPR,SHF,SYN     14
DEL,SYN         13
ADD,LPR         12
SHF             11
ADD,DEL         11
ADD,SHF,SYN     10
DEL,SHF         10
ADD,SYN         10
ADD,DEL,SYN      9
DEL,LPR,SYN      8
ADD,SHF          7
Name: obfuscation, dtype: int64